In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
flagD = 1
th = 200
ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
genedata, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

In [ ]:
Vn = 1.75E-12
Vc = 5.25E-12
traildoses = [0.000385, 0.001925, 0.00385, 0.01925, 0.0385, 0.1925, 0.385, 1.9250, 3.85, 19.25, 38.5]

startTime = datetime.now()
print(startTime)
spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
conds = [] # list for different conditions to be simulated
ttds = [] # list to save time-to-death values
deathcond = [] # list to save if the cell died or not
tout_all = np.arange(0,th*3600+1,30)
namexls = 'Fig3G_sims_TRAILdoses_200hr_det_'
for ii in range(len(traildoses)): 
    spdata2 = spdata
    spdata2[83] = traildoses[ii]
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata2
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model, solver) # Run simulation
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            xoutS_all[qq+1,:] = rdata['x'][-1,:]
            deathcond.append([1])
            break
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        deathcond.append([0])
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    conds.append(xoutS_all)
    ttds.append((qq+1)*ts)
    print(datetime.now() - startTime)
    condsDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
    condsDF.to_csv(namexls+str(ii)+'.txt',sep="\t")    
    print(datetime.now() - startTime)
ttsDF = pd.DataFrame(data=ttds) 
ttsDF.to_csv(namexls+'ttds.txt',sep="\t")
print(datetime.now())

In [ ]:
conds = []
ttds = []
namexls = 'Fig3G_sims_TRAILdoses_200hr_det_'
for ii in range(11):
    condsDF = pd.read_csv(namexls+str(ii)+'.txt',header=0,index_col=0,sep="\t")
    condsDF = np.double(condsDF.values[:])
    condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
    conds.append(condsDF)
    ttd = tout_all[condsDF.shape[0]-1]
    ttds.append(ttd)
    condsDF = None
    print(ii)

In [ ]:
plt.figure(figsize=(7, 4))
for ii in range(1,11,2):
    tout = np.arange(0,ttds[ii]+1,ts)
    plt.plot(tout/3600.0,(((conds[ii][:,105]))),linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('cPARP (nM)', multialignment='center')
plt.grid(True)
plt.xlim(0, 72)
plt.ylim(0, 400)
plt.xticks(np.arange(0,72,step=6))
plt.savefig('cPARP_Fig3G2.png')

In [ ]:
# traildoses = np.array([0.000385, 0.001925, 0.00385, 0.01925, 0.0385, 0.1925, 0.385, 1.9250, 3.85, 19.25, 38.5])
dosesngperml = traildoses*2.597402597402597e+01
plt.figure(figsize=(7, 4))
plt.plot(np.log10(dosesngperml),np.array(ttds)/3600.0,marker='s',linewidth=4)
plt.xlabel('log10(TRAIL dose)')
plt.ylabel('ttd (hrs)', multialignment='center')
plt.grid(True)
plt.savefig('cPARP_Fig3G3.png')